<a href="https://colab.research.google.com/github/Divyaanshvats/vats_AI_Gen_Prompt/blob/main/TASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MATHONGO AI PROMPT ENGINEERING TASK**

In [17]:
!pip install FPDF

# IMPORT ALL THE REQUIREMENTS

In [18]:
import json
import openai
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF
import os
import zipfile

# API HANDLING

In [19]:
client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key="gsk_XXXXXXXXXXXXXXXXXXXXXXXXXXXX" # I HAVE USED THE API KEY FOR GroqCloud
)

# LOAD THE JSON FILE

In [20]:
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data[0]

# Extracting and Summarizing Student Subject Performance

In [21]:
def extract_summary(data):
    subject_map = {
        "607018ee404ae53194e73d92": "Physics",
        "607018ee404ae53194e73d90": "Chemistry",
        "607018ee404ae53194e73d91": "Maths"
    }
    summary = []
    for sub in data["subjects"]:
        s_id = sub["subjectId"]["$oid"]
        summary.append({
            "subject": subject_map.get(s_id, s_id),
            "score": sub["totalMarkScored"],
            "accuracy": sub["accuracy"],
            "time_taken": sub["totalTimeTaken"] // 60  # we are dividing by 60 so to calculate in minutes
        })
    return summary

#  FEEDBACK GENERATION AND VISUALIZATION

In [22]:
def generate_feedback(summary, total_accuracy, total_score):
    prompt = f"""
You are an educational performance analyst. Analyze the following student test results:

Subject-wise details:
{json.dumps(summary, indent=2)}

Overall:
Total Score: {total_score}/300
Average Accuracy: {total_accuracy:.2f}%

Generate a PDF-ready feedback report with the following:
1. A subject-wise breakdown with observations (mention scoring trend, accuracy vs time).
2. Clear identification of strengths and weaknesses.
3. Explain time management — highlight if student rushed or overspent time.
4. Suggest 2–3 personalized, actionable steps per subject.
5. Conclude with an encouraging motivational message.

Be specific, analytical, and use natural human tone.
"""
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": "You are a helpful education assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

def plot_time_vs_accuracy(summary, plot_path):
    df = pd.DataFrame(summary)
    sns.set(style="whitegrid")
    plt.figure(figsize=(6, 4))
    sns.barplot(x="subject", y="accuracy", data=df, hue="subject", legend=False, palette="viridis")
    plt.ylabel("Accuracy (%)")
    plt.xlabel("Subject")
    plt.title("Accuracy per Subject")
    plt.tight_layout()
    plt.savefig(plot_path)
    plt.close()

# Personalized Student Feedback PDF

In [23]:
class FeedbackPDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 14)
        self.set_fill_color(230, 240, 255)
        self.cell(0, 10, 'AI-Powered Student Feedback Report', ln=True, align='C', fill=True)
        self.ln(10)

    def chapter_title(self, label):
        self.set_fill_color(200, 220, 255)
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, label, ln=True, fill=True)
        self.ln(2)

    def chapter_body(self, text):
        self.set_font('Arial', '', 11)
        self.multi_cell(0, 8, text)
        self.ln()

    def add_feedback(self, title, content):
        self.chapter_title(title)
        self.chapter_body(content)

def clean_text_for_pdf(text):
    return text.encode('latin-1', 'replace').decode('latin-1')

def process_file(json_path, output_pdf):
    data = load_json(json_path)
    summary = extract_summary(data)
    total_score = sum(sub["totalMarkScored"] for sub in data["subjects"])
    total_accuracy = sum(sub["accuracy"] for sub in data["subjects"]) / len(data["subjects"])

    feedback = generate_feedback(summary, total_accuracy, total_score)
    feedback = clean_text_for_pdf(feedback)  # ✅ Fix unicode issues

    plot_path = f"plot_{os.path.basename(json_path).split('.')[0]}.png"
    plot_time_vs_accuracy(summary, plot_path)

    pdf = FeedbackPDF()
    pdf.add_page()
    pdf.chapter_title("Performance Summary")
    summary_text = f"Total Score: {total_score}/300\nAverage Accuracy: {total_accuracy:.2f}%\n"
    pdf.chapter_body(summary_text)
    pdf.add_feedback("Personalized Feedback", feedback)
    pdf.image(plot_path, x=10, y=None, w=180)
    pdf.output(output_pdf)
    print(f"✅ Generated: {output_pdf}")

# ZIP THE PDFs

In [24]:
def zip_pdfs(pdf_files, zip_name="student_reports.zip"):
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for pdf in pdf_files:
            zipf.write(pdf)
    print(f"📦 Zipped into: {zip_name}")

# REPORT GENERATION

In [25]:
# --- Run on All Files ---
files = [
    "sample_submission_analysis_1.json",
    "sample_submission_analysis_2.json",
    "sample_submission_analysis_3.json",
    "sample_submission_analysis_3 (1).json"
]

pdf_files = []

for i, file in enumerate(files, start=1):
    try:
        output_name = f"student_feedback_report_{i}.pdf"
        process_file(file, output_name)
        pdf_files.append(output_name)
    except Exception as e:
        print(f"❌ Error in {file}: {e}")

✅ Generated: student_feedback_report_1.pdf
✅ Generated: student_feedback_report_2.pdf
✅ Generated: student_feedback_report_3.pdf
✅ Generated: student_feedback_report_4.pdf


# **GENERATE students_report.zip**

In [26]:
# --- Zip Output PDFs ---
zip_pdfs(pdf_files)

📦 Zipped into: student_reports.zip
